# 程式介紹：
## 欲解決之問題：
期望將youbike1.0與youbike2.0之站點做結合

## 內容介紹：
此程式主要為運用政府公開資訊Youbike 的站點搭配套件folium將站點資訊放入地圖中
最後結合區域邊界和站點資料，可以看看各地區的站點分佈
(要另外安裝geopandas&folium)

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import folium

## 讀入台北新北的資料

In [3]:
# 讀入新北資料
dfnew=pd.read_csv("https://quality.data.gov.tw/dq_download_csv.php?nid=28318&md5_url=45daed3330c8be0e1174e2f17b961d0a")
dfnew = dfnew[['sna','sarea','lat','lng','ar']]
dfnew['num'] = "1.0"

In [4]:
#讀入台北市2.0
dftaipei2 = pd.read_json("https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json")
dftaipei2 = dftaipei2[["sna","sarea","lat","lng","ar"]]
dftaipei2['num'] = "2.0"

In [5]:
# 讀入台北市的資料
dftaipei = pd.read_json("https://tcgbusfs.blob.core.windows.net/blobyoubike/YouBikeTP.json")

# 由json轉乘dataframe 分別用迴圈取出要的資訊
sna,lat,lng,sarea,ar=[],[],[],[],[]
for key,value in dftaipei["retVal"].iteritems():
    sna.append(value["sna"])
    lat.append(value["lat"])
    lng.append(value["lng"])
    sarea.append(value["sarea"])
    ar.append(value["ar"])
    
taipeidata={"sna":sna,"sarea":sarea,"lat":lat,"lng":lng,"ar":ar}
dftaipei = pd.DataFrame(taipeidata)
dftaipei['num'] = '1.0'

In [6]:
dfall=pd.concat([dftaipei,dftaipei2,dfnew])

In [7]:
dfall = dfall.reset_index(drop = True)

In [8]:
dfall[['lat', 'lng']].astype(float)

,lat,lng
0,25.040858,121.567904
1,25.041254,121.557420
2,25.037797,121.565169
3,25.036036,121.562325
4,25.036564,121.568664
...,...,...
1103,25.239000,121.632340
1104,25.017040,121.418210
1105,24.993520,121.504720
1106,25.072310,121.377520


## 執行主程式

In [10]:
place=input("請輸入區域：(如：信義區or汐止區or新店區)")
p = list(dfall.sarea.drop_duplicates())#取得所有地區區域名稱

while place not in p:
    place=input("您輸入的地區我們目前找不到，請試看看輸入完整名稱，如：信義區或板橋區")
    
else:
    la = dfall[dfall.sarea.isin([place])]['lat'].astype(float).mean()
    ln = dfall[dfall.sarea.isin([place])]['lng'].astype(float).mean()
    map1 = folium.Map([la,ln],zoom_start=13)
    search = dfall[dfall.sarea.isin([place])].values
    
for i in range(len(search)):
    if search[i][5]=='1.0':
        name = folium.Popup(search[i][4],max_width=300,min_width=300)
        folium.Marker([search[i][2],search[i][3]],popup=name,tooltip=search[i][0]).add_to(map1)
    else:
        name = folium.Popup(search[i][4],max_width=300,min_width=300)
        folium.Marker([search[i][2],search[i][3]],popup=name,tooltip=search[i][0],\
                      icon=folium.Icon(color='red')).add_to(map1)
map1

請輸入區域：(如：信義區or汐止區or新店區)大安區
